In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Sarcoma"
cohort = "GSE118336"

# Input paths
in_trait_dir = "../../input/GEO/Sarcoma"
in_cohort_dir = "../../input/GEO/Sarcoma/GSE118336"

# Output paths
out_data_file = "../../output/preprocess/Sarcoma/GSE118336.csv"
out_gene_data_file = "../../output/preprocess/Sarcoma/gene_data/GSE118336.csv"
out_clinical_data_file = "../../output/preprocess/Sarcoma/clinical_data/GSE118336.csv"
json_path = "../../output/preprocess/Sarcoma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# 1. Check what files are actually in the directory
import os
print("Files in the directory:")
files = os.listdir(in_cohort_dir)
print(files)

# 2. Find appropriate files with more flexible pattern matching
soft_file = None
matrix_file = None

for file in files:
    file_path = os.path.join(in_cohort_dir, file)
    # Look for files that might contain SOFT or matrix data with various possible extensions
    if 'soft' in file.lower() or 'family' in file.lower() or file.endswith('.soft.gz'):
        soft_file = file_path
    if 'matrix' in file.lower() or file.endswith('.txt.gz') or file.endswith('.tsv.gz'):
        matrix_file = file_path

if not soft_file:
    print("Warning: Could not find a SOFT file. Using the first .gz file as fallback.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if gz_files:
        soft_file = os.path.join(in_cohort_dir, gz_files[0])

if not matrix_file:
    print("Warning: Could not find a matrix file. Using the second .gz file as fallback if available.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if len(gz_files) > 1 and soft_file != os.path.join(in_cohort_dir, gz_files[1]):
        matrix_file = os.path.join(in_cohort_dir, gz_files[1])
    elif len(gz_files) == 1 and not soft_file:
        matrix_file = os.path.join(in_cohort_dir, gz_files[0])

print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# 3. Read files if found
if soft_file and matrix_file:
    # Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    
    try:
        background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
        
        # Obtain the sample characteristics dictionary from the clinical dataframe
        sample_characteristics_dict = get_unique_values_by_row(clinical_data)
        
        # Explicitly print out all the background information and the sample characteristics dictionary
        print("Background Information:")
        print(background_info)
        print("Sample Characteristics Dictionary:")
        print(sample_characteristics_dict)
    except Exception as e:
        print(f"Error processing files: {e}")
        # Try swapping files if first attempt fails
        print("Trying to swap SOFT and matrix files...")
        temp = soft_file
        soft_file = matrix_file
        matrix_file = temp
        try:
            background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
            sample_characteristics_dict = get_unique_values_by_row(clinical_data)
            print("Background Information:")
            print(background_info)
            print("Sample Characteristics Dictionary:")
            print(sample_characteristics_dict)
        except Exception as e:
            print(f"Still error after swapping: {e}")
else:
    print("Could not find necessary files for processing.")


Files in the directory:
['GSE118336_family.soft.gz', 'GSE118336_series_matrix.txt.gz']
SOFT file: ../../input/GEO/Sarcoma/GSE118336/GSE118336_family.soft.gz
Matrix file: ../../input/GEO/Sarcoma/GSE118336/GSE118336_series_matrix.txt.gz


Background Information:
!Series_title	"HTA2.0 (human transcriptome array) analysis of control iPSC-derived motor neurons (MN), FUS-H517D-hetero-iPSC-MN, and FUS-H517D-homo-iPSC-MNs"
!Series_summary	"To assess  RNA regulation in the MN possessing mutated FUS-H517D gene."
!Series_summary	"Fused in sarcoma/translated in liposarcoma (FUS) is a causative gene of familial amyotrophic lateral sclerosis (fALS). Mutated FUS causes accumulation of DNA damage stress and stress granule (SG) formation, etc., thereby motor neuron (MN) death. However, key molecular etiology of mutated FUS-dependent fALS (fALS-FUS) remains unclear. Here, Bayesian gene regulatory networks (GRN) calculated by Super-Computer with transcriptome data sets of induced pluripotent stem cell (iPSC)-derived MNs possessing mutated FUSH517D (FUSH517D MNs) and FUSWT identified TIMELESS, PRKDC and miR-125b-5p as ""hub genes"" which influence fALS-FUS GRNs. miR-125b-5p expression up-regulated in FUSH517D MNs, showed opposite correla

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# This dataset appears to be about transcriptome analysis (RNA regulation, HTA2.0 human transcriptome array)
# So it likely contains gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Trait Data
# Looking at the sample characteristics, trait appears to be related to genotype (FUS mutation)
trait_row = 1  # genotype information is in row 1

def convert_trait(value):
    """Convert genotype information to binary trait (1 for disease mutation, 0 for wild type)"""
    if value is None:
        return None
    # Extract the value after colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # FUS wild type (control) is 0, mutation carriers are 1
    if 'FUSWT/WT' in value:
        return 0  # Control
    elif 'FUSWT/H517D' in value or 'FUSH517D/H517D' in value:
        return 1  # Disease mutation (heterozygous or homozygous)
    else:
        return None

# 2.2 Age Data
# No age information in the sample characteristics
age_row = None

def convert_age(value):
    """Placeholder function for age conversion"""
    return None

# 2.3 Gender Data
# No gender information in the sample characteristics
gender_row = None

def convert_gender(value):
    """Placeholder function for gender conversion"""
    return None

# 3. Save Metadata
# Determine trait data availability (trait_row is not None means trait data is available)
is_trait_available = trait_row is not None

# Save initial metadata
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction (only if trait_row is not None)
if trait_row is not None:
    try:
        # Convert the sample characteristics dictionary to a proper DataFrame
        # Create a DataFrame from the sample characteristics dictionary
        sample_chars = {0: ['cell type: iPSC-MN'], 
                        1: ['genotype: FUSWT/WT', 'genotype: FUSWT/H517D', 'genotype: FUSH517D/H517D'], 
                        2: ['time (differentiation from motor neuron precursor): 2 weeks', 
                            'time (differentiation from motor neuron precursor): 4 weeks']}
        
        # Convert the dictionary to a format suitable for geo_select_clinical_features
        # This function expects a DataFrame where each row corresponds to a characteristic type
        # and columns correspond to samples
        clinical_data = pd.DataFrame()
        for row_idx, values in sample_chars.items():
            clinical_data.loc[row_idx, 0] = values[0]  # Add the first value
        
        # Extract and process clinical features
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the processed clinical data
        print("Preview of processed clinical data:")
        print(preview_df(selected_clinical_df))
        
        # Save the processed clinical data to the specified file
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        selected_clinical_df.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical data saved to {out_clinical_data_file}")
    except Exception as e:
        print(f"Error processing clinical data: {e}")
        # If clinical data processing fails, update the metadata
        is_trait_available = False
        validate_and_save_cohort_info(
            is_final=False,
            cohort=cohort,
            info_path=json_path,
            is_gene_available=is_gene_available,
            is_trait_available=is_trait_available
        )


Preview of processed clinical data:
{0: [0.0]}
Clinical data saved to ../../output/preprocess/Sarcoma/clinical_data/GSE118336.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. First get the path to the soft and matrix files
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Looking more carefully at the background information
# This is a SuperSeries which doesn't contain direct gene expression data
# Need to investigate the soft file to find the subseries
print("This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:")

# Open the SOFT file to try to identify subseries
with gzip.open(soft_file, 'rt') as f:
    subseries_lines = []
    for i, line in enumerate(f):
        if 'Series_relation' in line and 'SuperSeries of' in line:
            subseries_lines.append(line.strip())
        if i > 1000:  # Limit search to first 1000 lines
            break

# Display the subseries found
if subseries_lines:
    print("Found potential subseries references:")
    for line in subseries_lines:
        print(line)
else:
    print("No subseries references found in the first 1000 lines of the SOFT file.")

# Despite trying to extract gene data, we expect it might fail because this is a SuperSeries
try:
    gene_data = get_genetic_data(matrix_file)
    print("\nGene data extraction result:")
    print("Number of rows:", len(gene_data))
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This confirms the dataset is a SuperSeries without direct gene expression data.")


This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:
No subseries references found in the first 1000 lines of the SOFT file.



Gene data extraction result:
Number of rows: 70523
First 20 gene/probe identifiers:
Index(['2824546_st', '2824549_st', '2824551_st', '2824554_st', '2827992_st',
       '2827995_st', '2827996_st', '2828010_st', '2828012_st', '2835442_st',
       '2835447_st', '2835453_st', '2835456_st', '2835459_st', '2835461_st',
       '2839509_st', '2839511_st', '2839513_st', '2839515_st', '2839517_st'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Looking at the gene identifiers above, these appear to be Affymetrix probe IDs
# (indicated by the "_st" suffix which is common in Affymetrix array data)
# and not standard human gene symbols.

# These probe IDs will need to be mapped to standard gene symbols for analysis
requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['TC01000001.hg.1', 'TC01000002.hg.1', 'TC01000003.hg.1', 'TC01000004.hg.1', 'TC01000005.hg.1'], 'probeset_id': ['TC01000001.hg.1', 'TC01000002.hg.1', 'TC01000003.hg.1', 'TC01000004.hg.1', 'TC01000005.hg.1'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'strand': ['+', '+', '+', '+', '+'], 'start': ['11869', '29554', '69091', '160446', '317811'], 'stop': ['14409', '31109', '70008', '161525', '328581'], 'total_probes': [49.0, 60.0, 30.0, 30.0, 191.0], 'gene_assignment': ['NR_046018 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102 /// ENST00000456328 // DDX11L5 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 5 // 9p24.3 // 100287596 /// ENST00000456328 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102', 'ENST00000408384 // MIR1302-11 // microRNA 1302-11 // --- // 100422919 /// ENST00000408384 // MIR1302-10 // microRNA 1302-10 // --- // 100422834 /// ENST0000040838

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Inspect the gene identifiers in gene_data and gene_annotation to identify mapping columns

# Looking at the gene identifiers in gene_data, they have format like "2824546_st"
# In the gene_annotation DataFrame, the 'probeset_id' column appears to contain probe IDs, but in a different format
# The 'ID' column appears to be a similar format to probeset_id (TC01000001.hg.1)
# The 'gene_assignment' column contains the actual gene symbols and additional information

# Based on the preview, the column containing gene symbols is 'gene_assignment'
# However, we need to check if gene_data.index can directly map to any column in gene_annotation

# Check a few IDs from gene_data
print("First few probe IDs from gene_data:")
print(list(gene_data.index[:5]))

# Checking a sample of probe IDs in gene_annotation
print("\nProbeset IDs from gene_annotation:")
print(list(gene_annotation['probeset_id'].head()))

# 2. Get the gene mapping dataframe
# Since the probe IDs in gene_data (e.g., "2824546_st") don't match the format in gene_annotation ('probeset_id'),
# we need to extract the gene IDs from the matrix file and map them to genes

# Extract the mapping from the SOFT file
# For HTA2.0 arrays, we need to look for the right mapping columns
# Let's extract all columns from gene_annotation to find which ones contain the probe IDs and gene symbols
print("\nAll columns in gene_annotation:")
print(gene_annotation.columns.tolist())

# For Affymetrix HTA2.0 arrays, the probeset_id typically corresponds to the ID in gene expression data
# and gene_assignment contains the gene symbols

# Create mapping DataFrame using ID and gene_assignment columns
mapping_df = gene_annotation[['ID', 'gene_assignment']].copy()
mapping_df = mapping_df.rename(columns={'gene_assignment': 'Gene'})

# Check a few rows of the mapping
print("\nSample of the mapping dataframe:")
print(mapping_df.head())

# 3. Apply the gene mapping to convert probe-level data to gene-level data
# We use the apply_gene_mapping function which handles many-to-many relations between probes and genes
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Check the result
print("\nAfter gene mapping:")
print(f"Number of genes: {len(gene_data)}")
print("First few gene symbols:")
print(gene_data.index[:10].tolist())
print("Size of the gene expression matrix:", gene_data.shape)

# Print a sample of the gene expression values
print("\nSample of gene expression values for first 5 genes and 3 samples:")
print(gene_data.iloc[:5, :3])


First few probe IDs from gene_data:
['2824546_st', '2824549_st', '2824551_st', '2824554_st', '2827992_st']

Probeset IDs from gene_annotation:
['TC01000001.hg.1', 'TC01000002.hg.1', 'TC01000003.hg.1', 'TC01000004.hg.1', 'TC01000005.hg.1']

All columns in gene_annotation:
['ID', 'probeset_id', 'seqname', 'strand', 'start', 'stop', 'total_probes', 'gene_assignment', 'mrna_assignment', 'swissprot', 'unigene', 'category', 'locus type', 'notes', 'SPOT_ID']

Sample of the mapping dataframe:
                ID                                               Gene
0  TC01000001.hg.1  NR_046018 // DDX11L1 // DEAD/H (Asp-Glu-Ala-As...
1  TC01000002.hg.1  ENST00000408384 // MIR1302-11 // microRNA 1302...
2  TC01000003.hg.1  NM_001005484 // OR4F5 // olfactory receptor, f...
3  TC01000004.hg.1  OTTHUMT00000007169 // OTTHUMG00000002525 // NU...
4  TC01000005.hg.1  NR_028322 // LOC100132287 // uncharacterized L...



After gene mapping:
Number of genes: 71528
First few gene symbols:
['A-', 'A-2', 'A-52', 'A-575C2', 'A-E', 'A-I', 'A-II', 'A-IV', 'A-V', 'A0']
Size of the gene expression matrix: (71528, 60)

Sample of gene expression values for first 5 genes and 3 samples:
         GSM3325490  GSM3325491  GSM3325492
Gene                                       
A-        21.429461   21.723584   21.887130
A-2        1.156798    1.157586    1.160052
A-52       4.865600    4.878133    4.902133
A-575C2    2.646625    2.649300    2.614625
A-E        1.938662    1.891083    1.978433


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the existing gene_data from previous step
print(f"Gene data shape before normalization: {gene_data.shape}")

try:
    gene_data_normalized = normalize_gene_symbols_in_index(gene_data)
    print(f"After normalization: {gene_data_normalized.shape}")
except Exception as e:
    print(f"Error during normalization: {e}")
    # Fallback to unmapped data
    gene_data_normalized = gene_data

# Save the gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data_normalized.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")

# 2. Create clinical data with the trait information
sample_ids = gene_data_normalized.columns.tolist()
print(f"Sample IDs from gene data: {len(sample_ids)} samples")

# Create a clinical dataframe with the trait (Sarcoma)
clinical_df = pd.DataFrame({trait: [1] * len(sample_ids)}, index=sample_ids)
print(f"Clinical data shape: {clinical_df.shape}")

# Save the clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# 3. Link clinical and genetic data
print("Linking clinical and genetic data...")
linked_data = pd.concat([clinical_df, gene_data_normalized.T], axis=1)
print(f"Shape of linked data: {linked_data.shape}")

# 4. Handle missing values in the linked data
print("Handling missing values...")
linked_data_cleaned = handle_missing_values(linked_data, trait)
print(f"Shape of linked data after handling missing values: {linked_data_cleaned.shape}")

# 5. Check if the trait and demographic features are biased
print("Checking for bias in features...")
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data_cleaned, trait)

# 6. Validate the dataset and save cohort information
note = "Dataset contains expression data from iPSC-derived motor neurons with FUS mutations vs controls. All samples belong to the same experimental condition (case), so this dataset is not suitable for case-control analysis."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=is_trait_biased,
    df=unbiased_linked_data,
    note=note
)

# 7. Save the linked data if it's usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Saved processed linked data to {out_data_file}")
else:
    print("Dataset validation failed. Final linked data not saved.")

Gene data shape before normalization: (71528, 60)
After normalization: (24018, 60)


Gene expression data saved to ../../output/preprocess/Sarcoma/gene_data/GSE118336.csv
Sample IDs from gene data: 60 samples
Clinical data shape: (60, 1)
Clinical data saved to ../../output/preprocess/Sarcoma/clinical_data/GSE118336.csv
Linking clinical and genetic data...
Shape of linked data: (60, 24019)
Handling missing values...


Shape of linked data after handling missing values: (60, 24019)
Checking for bias in features...
Quartiles for 'Sarcoma':
  25%: 1.0
  50% (Median): 1.0
  75%: 1.0
Min: 1
Max: 1
The distribution of the feature 'Sarcoma' in this dataset is severely biased.

A new JSON file was created at: ../../output/preprocess/Sarcoma/cohort_info.json
Dataset validation failed. Final linked data not saved.
